In [139]:
import pandas as pd
import plotly.express as px
import numpy as np

In [109]:
annual = pd.read_csv("../Data/Annual_processed.csv", sep=",")
annual0 = pd.read_csv("../Data/Annual.csv", sep=';',decimal=',')
Export_total = pd.read_csv("../Data/Export_total.csv")
codes_pays=pd.read_csv("../Data/codes_pays.csv",sep=",")

In [110]:
codes_pays

,pays,iso_alpha3
0,AFGHANISTAN,AFG
1,AFRIQUE DU SUD,ZAF
2,ALBANIE,ALB
3,ALGERIE,DZA
4,ALLEMAGNE,DEU
...,...,...
214,WALLIS ET FUTUNA,WLF
215,YEMEN,YEM
216,YOUGOUSLAVIE,YUG
217,ZAMBIE,ZMB


In [167]:
pays_exports=annual[(annual['Libellé du flux']=="Exportations FAB") | (annual['Libellé du flux']=="Importations CAF")]
annualdf=pays_exports.set_index(['Libellé du pays','Libellé du flux'])
annualdf=annualdf.transpose().set_index(pd.date_range(start='2010-01-01', periods=11, freq='Y'))
annualdf=annualdf.unstack()
annualdf=annualdf.to_frame()[:]
annualdf=annualdf.reset_index()
fig = px.line(annualdf[(annualdf['Libellé du pays']=='ESPAGNE')], x='level_2', y=0, color='Libellé du flux')
fig.show()

In [163]:
print(Export_total['Libellé de la section CTCI'].nunique(),'\n',
      Export_total['Libellé de la division CTCI'].nunique(),'\n',
      Export_total['Libellé du groupe CTCI'].nunique(),'\n')

11 
 66 
 258 



le nombre des divisions et des groupes de CTCI est très grand, on les visualise pas 

In [169]:
continent_import =Export_total[(Export_total["Libellé du flux"] == "Importations CAF")].groupby(["Continent"]).mean().sort_values("Total dh", ascending=False)
continent_export =Export_total[(Export_total["Libellé du flux"] == "Exportations FAB")].groupby(["Continent"]).mean().sort_values("Total dh", ascending=False)
utilisation_import =Export_total[(Export_total["Libellé du flux"] == "Importations CAF")].groupby(["Libellé du groupement d'utilisation"]).mean().sort_values("Total dh", ascending=False)
utilisation_export =Export_total[(Export_total["Libellé du flux"] == "Exportations FAB")].groupby(["Libellé du groupement d'utilisation"]).mean().sort_values("Total dh", ascending=False)
section_export =Export_total[(Export_total["Libellé du flux"] == "Importations CAF")].groupby(["Libellé de la section CTCI"]).mean().sort_values("Total dh", ascending=False)
section_import =Export_total[(Export_total["Libellé du flux"] == "Exportations FAB")].groupby(["Libellé de la section CTCI"]).mean().sort_values("Total dh", ascending=False)

In [113]:
continent_import.head(2)

,Total dh
Continent,
AUTRE,3.206304e+08
EUROPE,8.742260e+07


In [114]:
fig = px.pie(continent_import, values='Total dh', names=continent_import.index, title='Partition des importations par continents(moyen des 3 derniers années)')
fig.show()

In [115]:
continent_export.head(2)

,Total dh
Continent,
EUROPE,1.122606e+08
AMERIQUE,5.786529e+07


In [116]:
utilisation_import.head(2)

,Total dh
Libellé du groupement d'utilisation,
ENERGIE ET LUBRIFIANTS,9.137951e+08
PRODUITS BRUTS D'ORIGINE MINERALE,8.162045e+07


In [117]:
utilisation_export.head(2)

,Total dh
Libellé du groupement d'utilisation,
OR INDUSTRIEL,3.228980e+08
"ALIMENTATION, BOISSONS ET TABACS",8.430104e+07


In [177]:
section_export.head(2)

,Total dh
Libellé de la section CTCI,
"COMBUSTIBLES ET MINÉRAUX, LUBRIFIANTS ET PRODUITS CONNEXES",8.583685e+08
MACHINES ET MATÉRIEL DE TRANSPORT,8.920993e+07


In [178]:
section_import.head(2)

,Total dh
Libellé de la section CTCI,
ARTICLES ET OPÉRATIONS NON CLASSÉS AILLEURS DANS LA CTCI,3.228980e+08
"PRODUITS CHIMIQUES ET PRODUITS CONNEXES, N.D.A.",8.831930e+07


let's concatenate them

In [180]:
sections=df = pd.DataFrame()
sections['total imports']=section_import['Total dh']
sections['total exports']=section_export['Total dh']
sections

,total imports,total exports
Libellé de la section CTCI,,
ARTICLES ET OPÉRATIONS NON CLASSÉS AILLEURS DANS LA CTCI,3.228980e+08,5.784906e+07
"PRODUITS CHIMIQUES ET PRODUITS CONNEXES, N.D.A.",8.831930e+07,7.100517e+07
PRODUITS ALIMENTAIRES ET ANIMAUX VIVANTS,8.781600e+07,7.863431e+07
MACHINES ET MATÉRIEL DE TRANSPORT,7.661401e+07,8.920993e+07
"COMBUSTIBLES ET MINÉRAUX, LUBRIFIANTS ET PRODUITS CONNEXES",7.270773e+07,8.583685e+08
MATIÈRES BRUTES NON COMESTIBLES À L'EXCEPTION DES CARBURANTS,4.728654e+07,4.372929e+07
ARTICLES MANUFACTURÉS DIVERS,3.559263e+07,2.477809e+07
BOISSONS ET TABACS,2.372767e+07,5.232274e+07
"HUILES, GRAISSES ET CIRES D'ORIGINE ANIMALE OU VÉGÉTALE",2.326177e+07,7.794036e+07


To draw a map ( choropleth) we have to get a dataset for exportations and importations , we can do that by filetreing or grouping by 'Libellé de flux', unfortunately we cannot do that with our dataset since the number of import rows and export rows is differents (they can't be columns in same dataframe).
So we will creat separately 2 Dataframes 

In [120]:
for_map0=annual[(annual['Libellé du flux']=="Exportations FAB")] # pour les exportations
for_map1=annual[(annual['Libellé du flux']=="Importations CAF")] # pour les importations
for_map0.drop('Libellé du flux',1,inplace=True)
for_map1.drop('Libellé du flux',1,inplace=True)
for_map0.head(2)

C:\Users\hp\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Libellé du pays,Valeur DHS 2010,Valeur DHS 2011,Valeur DHS 2012,Valeur DHS 2013,Valeur DHS 2014,Valeur DHS 2015,Valeur DHS 2016,Valeur DHS 2017,Valeur DHS 2018,Valeur DHS 2019,Valeur DHS 2020
0,ANDORRE,17000,0.0,1.132031e+06,3.086862e+06,6.370835e+06,7.754182e+06,7.308320e+06,5.987805e+06,6.375642e+06,6.127175e+06,3.778609e+06
8,EMIRATS ARABES UNIS,1259173840,741667974.0,4.931654e+08,4.322840e+08,5.882424e+08,5.366435e+08,9.631746e+08,9.328481e+08,4.943851e+08,8.604109e+08,8.737911e+08


In [121]:
for_map1.head(2)

,Libellé du pays,Valeur DHS 2010,Valeur DHS 2011,Valeur DHS 2012,Valeur DHS 2013,Valeur DHS 2014,Valeur DHS 2015,Valeur DHS 2016,Valeur DHS 2017,Valeur DHS 2018,Valeur DHS 2019,Valeur DHS 2020
4,ANDORRE,22162,1.015130e+05,1.262700e+04,3.052800e+04,6.468500e+04,1.431872e+06,1.038407e+06,3.561780e+05,1.218690e+05,7.419000e+03,1.450000e+02
6,EMIRATS ARABES UNIS,1644282647,2.460396e+09,4.508353e+09,2.572188e+09,3.605778e+09,5.314554e+09,6.509089e+09,5.069679e+09,7.686350e+09,8.720236e+09,4.850364e+09


when we filter some rows the index is no more a range of integers, let's set it again

In [122]:
for_map0.index=range(len(for_map0))
for_map1.index=range(len(for_map1))
print(for_map0.shape, for_map1.shape)

(217, 12) (234, 12)


let's create a column for the mean of values between 2010 and 2020

In [125]:
for_map0["Valeur moyen des exportations sur 10ans"]=for_map0[for_map0.columns[1:]].mean(axis=1)
for_map1["Valeur moyen des importations sur 10ans"]=for_map1[for_map1.columns[1:]].mean(axis=1)

C:\Users\hp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [126]:
for_map0.head(2)

,Libellé du pays,Valeur DHS 2010,Valeur DHS 2011,Valeur DHS 2012,Valeur DHS 2013,Valeur DHS 2014,Valeur DHS 2015,Valeur DHS 2016,Valeur DHS 2017,Valeur DHS 2018,Valeur DHS 2019,Valeur DHS 2020,Valeur moyen des exportations sur 10ans
0,ANDORRE,17000,0.0,1.132031e+06,3.086862e+06,6.370835e+06,7.754182e+06,7.308320e+06,5.987805e+06,6.375642e+06,6.127175e+06,3.778609e+06,4.358042e+06
1,EMIRATS ARABES UNIS,1259173840,741667974.0,4.931654e+08,4.322840e+08,5.882424e+08,5.366435e+08,9.631746e+08,9.328481e+08,4.943851e+08,8.604109e+08,8.737911e+08,7.432534e+08


droping the useless cols

In [127]:
for_map0.drop(for_map0.columns[1:12],1,inplace=True)
for_map0.head(2)

C:\Users\hp\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Libellé du pays,Valeur moyen des exportations sur 10ans
0,ANDORRE,4.358042e+06
1,EMIRATS ARABES UNIS,7.432534e+08


the same with importations dataset

In [128]:
for_map1.drop(for_map1.columns[1:12],1,inplace=True)
for_map1.head(2)

C:\Users\hp\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Libellé du pays,Valeur moyen des importations sur 10ans
0,ANDORRE,2.897641e+05
1,EMIRATS ARABES UNIS,4.812843e+09


we will creat a new column to fill it by the ISO Alpha 3 ( each country have a specific code), this code will serve for drawing the map

In [129]:
for_map1['alpha-3']='newcol'
for_map0['alpha-3']='newcol'

C:\Users\hp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [130]:
for_map1.head()

,Libellé du pays,Valeur moyen des importations sur 10ans,alpha-3
0,ANDORRE,2.897641e+05,newcol
1,EMIRATS ARABES UNIS,4.812843e+09,newcol
2,AFGHANISTAN,5.846716e+05,newcol
3,ANTIGUA ET BARBUDA,2.217273e+02,newcol
4,ANGUILLA,9.638182e+02,newcol


CHECHING to access the country name we should apply 
```
values()
```

In [133]:
print(for_map0['Libellé du pays'].values[1],'\n',
      for_map1['Libellé du pays'].values[1])

EMIRATS ARABES UNIS 
 EMIRATS ARABES UNIS


let's try getting the iso alpha 

In [ ]:
codes_pays[(codes_pays["pays"]=='EMIRATS ARABES UNIS')]['iso_alpha3']

52    ARE
Name: iso_alpha3, dtype: object

let's try to assign this isoalpha in the correspondant columns 

In [134]:
for_map0['alpha-3'][1]=codes_pays[(codes_pays["pays"]==for_map0['Libellé du pays'].values[1])]['iso_alpha3'].values[0]
for_map1['alpha-3'][1]=codes_pays[(codes_pays["pays"]==for_map1['Libellé du pays'].values[1])]['iso_alpha3'].values[0]

C:\Users\hp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hp\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



let's check

In [138]:
print('importations',for_map1[:3][:],'\n','exportations',for_map0[:3][:],'\n')

importations        Libellé du pays  Valeur moyen des importations sur 10ans alpha-3
0              ANDORRE                             2.897641e+05  newcol
1  EMIRATS ARABES UNIS                             4.812843e+09     ARE
2          AFGHANISTAN                             5.846716e+05  newcol 
 exportations        Libellé du pays  Valeur moyen des exportations sur 10ans alpha-3
0              ANDORRE                             4.358042e+06  newcol
1  EMIRATS ARABES UNIS                             7.432534e+08     ARE
2          AFGHANISTAN                             3.592146e+06  newcol 



well done! , in next cell i'll create a function to fill all iso alpha rows

In [140]:
def fill_iso_alpha(for_map):
    for i in range(len(for_map)):
        x=codes_pays[(codes_pays["pays"]==for_map['Libellé du pays'].values[i])]['iso_alpha3']
        if(len(x)!=0):
            for_map['alpha-3'][i]=codes_pays[(codes_pays["pays"]==for_map['Libellé du pays'].values[i])]['iso_alpha3'].values[0]
        else:
            for_map['alpha-3'][i]=np.nan

In [141]:
fill_iso_alpha(for_map0)
fill_iso_alpha(for_map1)

C:\Users\hp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hp\AppData\Local\Continuum\a

let's have a look

In [142]:
for_map0.head()

,Libellé du pays,Valeur moyen des exportations sur 10ans,alpha-3
0,ANDORRE,4.358042e+06,AND
1,EMIRATS ARABES UNIS,7.432534e+08,ARE
2,AFGHANISTAN,3.592146e+06,AFG
3,ANTIGUA ET BARBUDA,3.768104e+05,ATG
4,ANGUILLA,1.072427e+05,NaN


In [143]:
for_map0.isnull().sum()

Libellé du pays                            0
Valeur moyen des exportations sur 10ans    0
alpha-3                                    9
dtype: int64

In [144]:
for_map1.isnull().sum()

Libellé du pays                             0
Valeur moyen des importations sur 10ans     0
alpha-3                                    17
dtype: int64

oups! it bsems we have some messing data in iso alpha code (the represent 'autres pays '), we have nothing to do in this sense to replace them, so we will delete them :'(

In [145]:
for_map0.dropna(inplace=True)
for_map1.dropna(inplace=True)

C:\Users\hp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



AND FINALLY LET'S SAVE THOSE READY DATASETS AS CSV FOR USING LATER IN THE APP 

In [146]:
for_map0.to_csv('for_map0.csv',index=False)
for_map1.to_csv('for_map1.csv',index=False)

I will test to plot a map from the first datasqet

In [158]:

fig = go.Figure(data=go.Choropleth(
    locations = for_map0['alpha-3'],
    z = for_map0['Valeur moyen des exportations sur 10ans'],
    text = for_map0['Libellé du pays'],
    colorscale = ['#99004d', '#ff3287', '#fc76c4', '#ca89bd'],
    #colorscale='Blues',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='white',
    marker_line_width=1,
    colorbar_tickprefix = 'dh ',
    colorbar_title = 'Exportation en Dh',
))

fig.update_layout(
    title_text="Moyenne annuel d'exportation entre 2010 et 2020",
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=-0.1,
        xref='paper',
        yref='paper',
        text='Source: <a href="https://www.oc.gov.ma/">\
            Office des Changes Marocain</a>',
        showarrow = False
    )]
)
fig.update_layout(
    margin=dict(l=0, r=0, t=34, b=5),
    paper_bgcolor="rgb(180, 180, 180)")
fig.update_layout(geo=dict(bgcolor= 'rgba(0,0,0,0)'))
fig.update_layout(hovermode='closest',)
fig.show()